# Landau-Zener Model Simulator

### Abstract
This notebook presents the time evolution simulation of a qubit according to the Landau Zener (LZ) Hamiltonian on a real IBM Quantum Computer. Time evolution is discretized by performing small finite evolutions and the experimental probabilities of the $|0\rangle$ state are measured as a function of the simulation time. The circuit is implemented on the 5-qubit ibmq_lima computer, running the time evolution of each of the qubits in parallel.

### Introduction


### Useful Packages

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import csv
import qiskit
from qiskit import *
from qiskit.tools.monitor import job_monitor
from IPython.display import clear_output

### IBM Configuration

In [4]:
# Load IBM Qiskit Account
IBMQ.load_account();

# IBM Provider and Backends
provider = IBMQ.get_provider('ibm-q')
simulator = Aer.get_backend('qasm_simulator')
lima = provider.get_backend('ibmq_lima')

ibmqfactory.load_account:WARNING:2022-03-02 01:17:53,123: Credentials are already in use. The existing account in the session will be replaced.


### LZ Hamiltonian

The LZ Hamiltonian expressed in terms of the $\{|0\rangle,|1\rangle\}$ basis is

$$\hat{H} = \frac{1}{2}\pmatrix{\Delta t & \omega_0\\ \omega_0 & -\Delta t}$$

where $\omega_0$ is the energy bandgap at the avoided level crossing at $t=0$ and $\Delta$ is the rate of change of the time dependent components of the $\hat{H}$. The following figure shows the instantaneous energy spectrum of this Hamiltonian in time, solid lines show the case in which $\omega_0>0$ while dashed lines represent the decoupled case in which $\omega_0=0$.

![alt text](spectrum.svg "LZ Energy Spectrum")

The adiabatic theorem states that while the energy bandgap is big enough, the system will approximately remain on the instantaneous eigenstate of the Hamiltonian. Near the avoided level crossing at $t=0$, the system undergoes significant LZ transitions as a consequence of the bandgap reaching a minimum.

In [5]:
# LZ Hamiltonian
def H(t, w0, D):
    return 0.5*np.array([[D*t, w0], [w0, -D*t]])

### Time Evolution Simulation
In order to approximate the time evolution of a qubit according to the LZ Hamiltonian using a digital quantum computer, it is necessary to perform a discretization of the time evolution operator. Dividing the total time interval $T=T_f-T_0$ in $N$ points, the Hamiltonian can be evaluated in each point and the qubit is evolved as in a time independent case for a small interval $dt = T/N$ (i.e. the size of the steps).

![alt text](aprox.svg "Discretization")

Each exponential operator from the time independent cases can be included to the quantum circuit to simulate the time evolution as a product of unitary gates

$$\hat{U}(t) \approx \prod_{k=0}^{N-1}\exp{\left(-\frac{i\hat{H}(kdt)\>dt}{\hbar}\right)}$$

### Measurement Calibration
Readout errors are one of the main sources of noise in the experimental probabilities obtained. The results can be corrected using a calibration matrix $A$ which can be obtained from the backend properties:

* prob_meas1_prep0: probability of preparing state $|0\rangle$ and measuring state $|1\rangle$.
* prob_meas0_prep1: probability of preparing state $|1\rangle$ and measuring state $|0\rangle$.

Given a vector of noisy probabilities $P_{\text{noisy}}$ representing the experimental readouts and a vector of ideal probabilities $P_{\text{ideal}}$

$$P_{\text{noisy}} = AP_{\text{ideal}}$$

where

$$A = \pmatrix{1-\text{prob_meas1_prep0} & \text{prob_meas0_prep1}\\ \text{prob_meas1_prep0} & 1-\text{prob_meas0_prep1}}$$

finally, ideal probabilities can be recovered by inverting the calibration matrix

$$P_{\text{ideal}} = A^{-1}P_{\text{noisy}}$$

In [20]:
# Measurements per circuit
Nshots = 5000

# Initial indexes
iN = 0
iD = 0
iAct = 0
iActN = 0
iActD = 0

In [ ]:
# Anticrossing Energy Bandgap
w0 = 1

# Delta and N parameters list
Dlist = np.linspace(2, 20, 10)
Nlist = np.array([50, 100, 150, 200])

for iN in range(iActN, len(Nlist)):
    iActN = iN
    
    N = Nlist[iN]
    
    # Reset measurements matrix to 0
    if(iAct == 0):
        P0 = np.zeros((N, 5))
    
    for iD in range(iActD, len(Dlist)):
        iActD = iD
        D = Dlist[iD]
        
        # Data filename
        filename = 'dataTemp_SH'+str(Nshots)+'_N'+str(Nlist[iActN])+'_w'+str(w0)+'_D'+str(Dlist[iActD])+'.dat'
        
        # Backend Properties
        props = lima.properties()
        
        file = open(filename,"w", newline='')
        
        # Calibration Matrices
        A = np.zeros((5, 2, 2))
        
        for q in range(len(props.qubits)):
            qi = props.qubits[q]
            file.write('Q'+str(q)+'\n')
            
            # Set Calibration Matrices
            prob_meas0_prep1 = qi[5].value
            prob_meas1_prep0 = qi[6].value
            A[q,:,:] = np.array([[1-prob_meas1_prep0, prob_meas0_prep1], [prob_meas1_prep0, 1-prob_meas0_prep1]])
            
            for i in range(len(qi)):
                data = str(qi[i].date.year)+','+str(qi[i].date.month)+','+str(qi[i].date.day)+','+str(qi[i].date.minute)+','+str(qi[i].date.second)+','+qi[i].name+','+str(qi[i].value)+','+qi[i].unit
                file.write(data)
                file.write('\n')
            file.write('\n')

        # Simulation Time
        T0 = 0
        Tf = 4
        T = Tf - T0
        dt = (T/N)
        t = np.linspace(T0,Tf,N)
        
        # Initial eigenstate calculation
        Htemp = H(T0, w0 = w0, D = D)
        
        eigVal = sp.linalg.eigvals(Htemp)
        eigVec = sp.linalg.eig(Htemp, left=False, right=True)[1]

        indBaseState = np.argmin(eigVal)
        initState = eigVec[:, indBaseState]
        
        for i in range(iAct, len(t)):
            iAct = i
            
            # Print Progress
            clear_output(wait=True)
            print('It: ', i+1, '/', N )
            print('------------------------------------')

            # Graph Real Time Measurements
            fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 8))
            
            ax[0,0].scatter(t[0:iAct], P0[0:iAct,0], label = 'q0', color = 'k', s = 5)
            ax[0,0].set_xlim((T0,Tf))
            ax[0,0].set_ylim((0,1))
            ax[0,0].set_title('Qubit 0 , '+'N='+str(Nlist[iActN])+r' , $\Delta$='+str(Dlist[iActD]))
            
            ax[0,1].scatter(t[0:iAct], P0[0:iAct,1], label = 'q1', color = 'r', s = 5)
            ax[0,1].set_xlim((T0,Tf))
            ax[0,1].set_ylim((0,1))
            ax[0,1].set_title('Qubit 1 , '+'N='+str(Nlist[iActN])+r' , $\Delta$='+str(Dlist[iActD]))
            
            ax[0,2].scatter(t[0:iAct], P0[0:iAct,2], label = 'q2', color = 'g', s = 5)
            ax[0,2].set_xlim((T0,Tf))
            ax[0,2].set_ylim((0,1))
            ax[0,2].set_title('Qubit 2 , '+'N='+str(Nlist[iActN])+r' , $\Delta$='+str(Dlist[iActD]))
            
            ax[1,0].scatter(t[0:iAct], P0[0:iAct,3], label = 'q3', color = 'b', s = 5)
            ax[1,0].set_xlim((T0,Tf))
            ax[1,0].set_ylim((0,1))
            ax[1,0].set_title('Qubit 3 , '+'N='+str(Nlist[iActN])+r' , $\Delta$='+str(Dlist[iActD]))
            
            ax[1,1].scatter(t[0:iAct], P0[0:iAct,4], label = 'q4', color = 'm', s = 5)
            ax[1,1].set_xlim((T0,Tf))
            ax[1,1].set_ylim((0,1))
            ax[1,1].set_title('Qubit 4 , '+'N='+str(Nlist[iActN])+r' , $\Delta$='+str(Dlist[iActD]))
            plt.show()

            # Quantum Circuit Creation
            qr = QuantumRegister(5, name = 'q')
            cr = ClassicalRegister(5, name = 'c0')
            circuit = QuantumCircuit(qr, cr)

            # Qubit Initial States
            circuit.initialize(initState, 0)
            circuit.initialize(initState, 1)
            circuit.initialize(initState, 2)
            circuit.initialize(initState, 3)
            circuit.initialize(initState, 4)
            
            circuit.barrier()

            # Add Time Evolution Gates
            for j in range(i):
                dU = sp.linalg.expm(-1j*dt*H(t[j], w0 = w0, D = D))
                circuit.unitary(dU, qr[0])
                circuit.unitary(dU, qr[1])
                circuit.unitary(dU, qr[2])
                circuit.unitary(dU, qr[3])
                circuit.unitary(dU, qr[4])


                if(j != (i-1)):
                    circuit.barrier()

            # Measurement Gate
            circuit.measure(qr, cr)

            # Run on IBM Quantum
            job = execute(circuit, backend = lima, shots = Nshots)
            job_monitor(job)

            # Experiment Results
            result = job.result()
            data = result.data()
            cnts = data.get('counts')

            cnts2 = np.zeros(31)
            for k in range(len(cnts2)):
                cnts2[k] = cnts.get(hex(k))
                if( np.isnan(cnts2[k]) ):
                    cnts2[k] = 0

            # Calculations of probability of state 0
            # Qubit 4
            P0[i,4] = (1/Nshots)*( cnts2[0] + cnts2[1] + cnts2[2] + cnts2[3] + cnts2[4]
                                  + cnts2[5] + cnts2[6] + cnts2[7] + cnts2[8] + cnts2[9]
                                  + cnts2[10] + cnts2[11] + cnts2[12] + cnts2[13] + cnts2[14]
                                  + cnts2[15] )
            # Qubit 3
            P0[i,3] = (1/Nshots)*( cnts2[0] + cnts2[1] + cnts2[2] + cnts2[3] + cnts2[4]
                                  + cnts2[5] + cnts2[6] + cnts2[7] + cnts2[16] + cnts2[17]
                                  + cnts2[18] + cnts2[19] + cnts2[20] + cnts2[21] + cnts2[22]
                                  + cnts2[23] )
            # Qubit 2
            P0[i,2] = (1/Nshots)*( cnts2[0] + cnts2[1] + cnts2[2] + cnts2[3] + cnts2[8] 
                                  + cnts2[9] + cnts2[10] + cnts2[11] + cnts2[16] + cnts2[17]
                                  + cnts2[18] + cnts2[19] + cnts2[24] + cnts2[25] + cnts2[26]
                                  + cnts2[27])
            # Qubit 1
            P0[i,1] = (1/Nshots)*( cnts2[0] + cnts2[1] + cnts2[4] + cnts2[5] + cnts2[8]
                                  + cnts2[9] + cnts2[12] + cnts2[13] + cnts2[16] + cnts2[17]
                                  + cnts2[20] + cnts2[21] + cnts2[24] + cnts2[25] + cnts2[28]
                                  + cnts2[29])
            # Qubit 0
            P0[i,0] = (1/Nshots)*( cnts2[0] + cnts2[2] + cnts2[4] + cnts2[6] + cnts2[8]
                                  + cnts2[10] + cnts2[12] + cnts2[14] + cnts2[16] + cnts2[18]
                                  + cnts2[20] + cnts2[22] + cnts2[24] + cnts2[26] + cnts2[28]
                                  + cnts2[30])
            
            for ii in range(5):
                invA = np.linalg.inv(A[ii,:,:])
                P0[i,ii] = invA[0,0]*P0[i,ii] + invA[0,1]*(1-P0[i,ii])
            
        temp = np.reshape(t, (len(t), 1))
        P0q0 = np.reshape(P0[:,0], (len(P0[:,0]), 1))
        P0q1 = np.reshape(P0[:,1], (len(P0[:,1]), 1))
        P0q2 = np.reshape(P0[:,2], (len(P0[:,2]), 1))
        P0q3 = np.reshape(P0[:,3], (len(P0[:,3]), 1))
        P0q4 = np.reshape(P0[:,4], (len(P0[:,4]), 1))

        data = np.hstack( ( temp, P0q0, P0q1, P0q2, P0q3, P0q4))

        header = np.array(['t', 'P0q0', 'P0q1', 'P0q2', 'P0q3', 'P0q4'])
        
        data = np.vstack((header, data))

        # Save data on file
        with file as f:
            csv.writer(f, delimiter=',').writerows(data)
            
        file.close()
        
        iAct = 0
    iActD = 0

It:  50 / 50
------------------------------------
